In [1]:
import ast
import numpy as np
import pandas as pd
from coverage_functions import *

# Filtering Homology Models wrt PDB structures 

In [2]:
modbase = pd.read_csv("processed_data/modbase/modbase_30aa_hq_fixed.tsv", sep="\t", dtype={'PDBBegin': 'object','PDBEnd': 'object'})
swissmodel = pd.read_csv("processed_data/swissmodel/swissmodel_30aa_hq.tsv", sep="\t")
swissmodel = swissmodel.rename(columns={'UniProtKB_ac': 'Entry', 'uniprot_seq_length': 'Length', 'from': 'TargetBeg', 'to': 'TargetEnd'})
print(len(modbase), len(swissmodel))

35808 9541


In [3]:
# filtered pdb structures for uniprot entries
pdb = pd.read_csv("processed_data/uniprot/proteome_have_pdb_begin_end_missing_consec_greater_30pdb.csv")
len(pdb)

126005

In [4]:
# get all covered residues for every entry (missings already removed)
def get_all_covered_residues(pdb_file):
    pdb_file = pdb_file.drop_duplicates(subset=["Entry", "ResidueList"], keep="last") # 51845
    pdb_file['ResidueList'] = pdb_file['ResidueList'].apply(lambda x: ast.literal_eval(x))
    pdb_file2 = pdb_file.groupby('Entry')['ResidueList'].apply(list).reset_index(name='AllCoveredResidues')
    pdb_file2['AllCoveredResidues'] = pdb_file2['AllCoveredResidues'].apply(lambda x: sum(x, [])).apply(set)
    return pdb_file2

pdb2 = get_all_covered_residues(pdb)

In [6]:
print(len(pdb2), pdb2.Entry.nunique(), pdb.Entry.nunique())

7085 7085 7085


In [7]:
def get_model_residues(beg, end):
    residues = list(range(int(beg), int(end)+1))
    return set(residues)

In [8]:
# get model residues
swissmodel['model_residues'] = swissmodel.apply(lambda x: get_model_residues(beg=x['TargetBeg'], end=x['TargetEnd']), axis=1)
modbase['model_residues'] = modbase.apply(lambda x: get_model_residues(beg=x['TargetBeg'], end=x['TargetEnd']), axis=1)

In [9]:
# merge with new pdb dataframe to add all covered residues by PDB
swissmodel = swissmodel.merge(pdb2, on='Entry', how='left')
modbase = modbase.merge(pdb2, on='Entry', how='left')

In [11]:
# there may be no PDB structures for some entries, store them into another dataframe
# swiss-model
swissmodel_NoPDB = swissmodel[swissmodel['AllCoveredResidues'].isnull()]
swissmodel_PDB = swissmodel[~swissmodel['AllCoveredResidues'].isnull()]
# modbase
modbase_NoPDB = modbase[modbase['AllCoveredResidues'].isnull()]
modbase_PDB = modbase[~modbase['AllCoveredResidues'].isnull()]

In [ ]:
# we need residues present in the model but not the pdb structure
# get the residues ONLY covered by the model
swissmodel_PDB['difference'] = swissmodel_PDB.apply(lambda x: x['model_residues']-x['AllCoveredResidues'], axis=1)
modbase_PDB['difference'] = modbase_PDB.apply(lambda x: x['model_residues']-x['AllCoveredResidues'], axis=1)

In [ ]:
# get the number of residues only covered by the model
swissmodel_PDB['only_in_model'] = swissmodel_PDB['difference'].apply(len)
modbase_PDB['only_in_model'] = modbase_PDB['difference'].apply(len)

In [16]:
print((len(swissmodel_PDB), len(modbase_PDB)), (len(swissmodel_NoPDB), len(modbase_NoPDB)))

(6421, 20424) (3120, 15384)


In [20]:
len(swissmodel_PDB[swissmodel_PDB['only_in_model'] >= 30]), len(swissmodel_PDB[swissmodel_PDB['only_in_model'] > 0])

(1225, 2810)

In [21]:
len(modbase_PDB[modbase_PDB['only_in_model'] >= 30]), len(modbase_PDB[modbase_PDB['only_in_model'] > 0])

(3570, 11707)

In [22]:
# get the models if the difference is greater than 0
swissmodel_PDB_greater_0 = swissmodel_PDB[swissmodel_PDB['only_in_model'] > 0]
modbase_PDB_greater_0 = modbase_PDB[modbase_PDB['only_in_model'] > 0]

In [23]:
# combine
swissmodel_ALL = pd.concat([swissmodel_PDB_greater_0, swissmodel_NoPDB], axis=0, ignore_index=True)
modbase_ALL = pd.concat([modbase_PDB_greater_0, modbase_NoPDB], axis=0, ignore_index=True)

In [24]:
print(len(swissmodel_ALL), len(swissmodel_PDB_greater_0), len(swissmodel_NoPDB), len(swissmodel_PDB_greater_0)+len(swissmodel_NoPDB))
print(len(modbase_ALL), len(modbase_PDB_greater_0), len(modbase_NoPDB), len(modbase_PDB_greater_0)+len(modbase_NoPDB))

5930 2810 3120 5930
27091 11707 15384 27091


In [37]:
# save to file
modbase_ALL.to_excel('processed_data/modbase/modbase_30aa_hq_fixed_extra_to_pdb.xlsx', index=False)
swissmodel_ALL.to_excel('processed_data/swissmodel/swissmodel_30aa_hq_extra_to_pdb.xlsx', index=False)